In [4]:
import sys, os
sys.path.insert(1,"/opt/lumerical/v221/api/python/") #Default linux lumapi path
#sys.path.append(os.path.dirname("/media/lucas/datam01/LucasLobo/Design_MMI_LucasLobo/python")) 
import  lumapi

In [5]:
mode = lumapi.MODE()